<a href="https://colab.research.google.com/github/Marwanelhanafey/Tensorflow-and-PyTorch/blob/main/Tensorflow%20and%20PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow_Pytorch_Lab

## Nessesary Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import tensorflow as tf
import time
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
pip onnx

ERROR: unknown command "onnx"


## Tensorflow


In [3]:
def run_tensorflow():
    # Load and preprocess MNIST data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train.reshape(-1, 784) / 255.0, x_test.reshape(-1, 784) / 255.0

    # Build model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train model, capture history to get loss
    start_train = time.time()
    history = model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=2)
    train_time = time.time() - start_train
    print(f"Training time (TF): {train_time:.2f} s")

    # Get final training loss from last epoch
    final_loss = history.history['loss'][-1]

    # Evaluate model
    start_eval = time.time()
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    infer_time = time.time() - start_eval
    print(f"Test Accuracy (TF): {acc:.4f}")
    print(f"Inference time (TF): {infer_time:.2f} s")

    # Export to TFLite
    tflite_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()
    tflite_path = "model.tflite"
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)

    # Get model size in MB
    model_size_mb = os.path.getsize(tflite_path) / (1024 * 1024)

    return train_time, infer_time, acc, model_size_mb, final_loss


## MNIST Model



In [4]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

## Pytorch

In [5]:
def run_pytorch():
    # Data loading and normalization
    transform = transforms.Compose([transforms.ToTensor()])
    train_data = datasets.MNIST(root='.', train=True, download=True, transform=transform)
    test_data = datasets.MNIST(root='.', train=False, transform=transform)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=1000)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MNISTModel().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    model.train()
    start_train = time.time()
    final_loss = None
    for epoch in range(5):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        final_loss = running_loss / len(train_loader)  # Average loss per batch for the last epoch
    train_time = time.time() - start_train
    print(f"Training time (PyTorch): {train_time:.2f} s")

    model.eval()
    correct = 0
    total = 0
    start_eval = time.time()
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    infer_time = time.time() - start_eval
    accuracy = correct / total
    print(f"Test Accuracy (PyTorch): {accuracy:.4f}")
    print(f"Inference time (PyTorch): {infer_time:.2f} s")

    dummy_input = torch.randn(1, 784, device=device)
    onnx_path = "model.onnx"
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
    )
    model_size_mb = os.path.getsize(onnx_path) / (1024 * 1024)

    return train_time, infer_time, accuracy, model_size_mb, final_loss


## Plotting

In [6]:
import numpy as np
import matplotlib.pyplot as plt

def plot_comprehensive(frameworks, training_time, inference_time, accuracy, loss, output_img='comparison_full_metrics.png'):
    x = np.arange(len(frameworks))
    width = 0.4

    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Training Time
    axs[0, 0].bar(x, training_time, width, color='skyblue')
    axs[0, 0].set_title('Training Time (s)')
    axs[0, 0].set_xticks(x)
    axs[0, 0].set_xticklabels(frameworks)
    for i, v in enumerate(training_time):
        axs[0, 0].text(i, v + 0.1, f"{v:.2f}", ha='center')

    # Inference Time
    axs[0, 1].bar(x, inference_time, width, color='lightgreen')
    axs[0, 1].set_title('Inference Time (s)')
    axs[0, 1].set_xticks(x)
    axs[0, 1].set_xticklabels(frameworks)
    for i, v in enumerate(inference_time):
        axs[0, 1].text(i, v + 0.01, f"{v:.3f}", ha='center')

    # Accuracy
    axs[1, 0].bar(x, accuracy, width, color='gold')
    axs[1, 0].set_title('Test Accuracy')
    axs[1, 0].set_ylim(0, 1)
    axs[1, 0].set_xticks(x)
    axs[1, 0].set_xticklabels(frameworks)
    for i, v in enumerate(accuracy):
        axs[1, 0].text(i, v + 0.02, f"{v:.4f}", ha='center')

    # Loss
    axs[1, 1].bar(x, loss, width, color='salmon')
    axs[1, 1].set_title('Final Training Loss')
    axs[1, 1].set_xticks(x)
    axs[1, 1].set_xticklabels(frameworks)
    for i, v in enumerate(loss):
        axs[1, 1].text(i, v + 0.02, f"{v:.4f}", ha='center')

    plt.tight_layout()
    fig.savefig(output_img, dpi=300)
    plt.close()

    print(f"Full comparison metrics saved to {output_img}")


## Results

In [7]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 115.3 MB/s eta 0:00:00


In [8]:
if __name__ == "__main__":
    print("Running TensorFlow MNIST model...")
    tf_train_time, tf_infer_time, tf_acc, tf_model_size, tf_loss = run_tensorflow()

    print("\nRunning PyTorch MNIST model...")
    pt_train_time, pt_infer_time, pt_acc, pt_model_size, pt_loss = run_pytorch()

    frameworks = ['TensorFlow', 'PyTorch']
    training_time = [tf_train_time, pt_train_time]
    inference_time = [tf_infer_time, pt_infer_time]
    accuracy = [tf_acc, pt_acc]
    loss = [tf_loss, pt_loss]

    plot_comprehensive(frameworks, training_time, inference_time, accuracy, loss)


Running TensorFlow MNIST model...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 - 9s - 5ms/step - accuracy: 0.9161 - loss: 0.3031
Epoch 2/5
1875/1875 - 7s - 4ms/step - accuracy: 0.9568 - loss: 0.1465
Epoch 3/5
1875/1875 - 5s - 3ms/step - accuracy: 0.9684 - loss: 0.1067
Epoch 4/5
1875/1875 - 4s - 2ms/step - accuracy: 0.9744 - loss: 0.0843
Epoch 5/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9798 - loss: 0.0675
Training time (TF): 32.31 s
Test Accuracy (TF): 0.9724
Inference time (TF): 1.48 s
Saved artifact at '/tmp/tmph9jyyfrn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138055391123152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138055391126608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138055391128528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138055391122768: TensorSpec(shape=(), dtype=tf.resource, name=None)

Running P

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.60MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.78MB/s]


Training time (PyTorch): 45.60 s
Test Accuracy (PyTorch): 0.9721
Inference time (PyTorch): 1.02 s
Full comparison metrics saved to comparison_full_metrics.png
